# Farm ponds recognition model training
This is a notebook to train a MaskRCNN model to recognize farm ponds in satellite images. The code is based on Detectron2 documentation. You can find more details of the pre-train models from [Meta Research's Github repository](https://github.com/facebookresearch/detectron2)

In [1]:
import torch, detectron2

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  2.0 ; cuda:  cu117
detectron2: 0.6


## Test on Kadwanchi Data

This is a test on of the trained model from Kadwanchi. To use a pre-trained model, you will need the config file of a pretrained model(yaml) and the model weights (pth). The pre-trained model needs pred_mask to color the area of the farm ponds. You can swap the test folder path to other image sets to test and visualize the prediction results.

In [3]:
from detectron2.utils.visualizer import ColorMode
from itertools import compress
from PIL import Image  
from skimage import io, color, morphology, segmentation
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the trained model 

#DetectionCheckpointer(model).load('/home/pcl1225/code/farmponds/output/model_999.pth') 
predictor = DefaultPredictor(cfg)
pond_metadata = MetadataCatalog.get("pond_train")


test_folder_path = '/home/pcl1225/code/farmponds/dataset/test_kadwanchi'
output_folder = '/home/pcl1225/code/farmponds/dataset/mask_test'
os.makedirs(output_folder, exist_ok=True)
image_files = [file for file in os.listdir(test_folder_path) if file.lower().endswith(('.png'))]

def cropper(org_image_path, mask_array, out_file_name):
    num_instances = mask_array.shape[0]
    mask_array = np.moveaxis(mask_array, 0, -1)
    mask_array_instance = []
    img = cv2.imread(str(org_image_path))
    output = np.zeros_like(img)
    for i in range(num_instances):
        mask_array_instance.append(mask_array[:, :, i:(i+1)])
        output = np.where(mask_array_instance[i] == True, 255, output)
    im = Image.fromarray(output)
    im.save(out_file_name)


# Loop through each image file
for d in image_files:
    im_path = os.path.join(test_folder_path, d)
    im = cv2.imread(im_path)
    outputs = predictor(im)
    masks = outputs['instances'].pred_masks
    masks= masks.to('cpu').numpy()
    #num, h, w= masks.shape
    mask_path = os.path.join(output_folder, d)
    cropper(im_path, masks, mask_path)
    

# Show three random samples
for d in random.sample(image_files, 3):
    im_path = os.path.join(test_folder_path, d)
    print(im_path)
    im = cv2.imread(im_path)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=pond_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()



[02/26 22:54:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://ImageNetPretrained/FAIR/X-101-32x8d.pkl ...


X-101-32x8d.pkl: 356MB [00:03, 107MB/s]                               


[02/26 22:54:13 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[02/26 22:54:14 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up:
| Names in Model    | Names in Checkpoint                         | Shapes                                          |
|:------------------|:--------------------------------------------|:------------------------------------------------|
| res2.0.conv1.*    | res2_0_branch2a_{bn_b,bn_rm,bn_riv,bn_s,w}  | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.0.conv2.*    | res2_0_branch2b_{bn_b,bn_rm,bn_riv,bn_s,w}  | (256,) (256,) (256,) (256,) (256,8,3,3)         |
| res2.0.conv3.*    | res2_0_branch2c_{bn_b,bn_rm,bn_riv,bn_s,w}  | (256,) (256,) (256,) (256,) (256,256,1,1)       |
| res2.0.shortcut.* | res2_0_branch1_{bn_b,bn_rm,bn_riv,bn_s,w}   | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.1.conv1.*    | res2_1_branch2a_{bn_b,bn_rm,bn_riv,bn_s,w}  | (256,) (256,) (256,) (256,) (256,2

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}
The checkpoint state_dict contains 

KeyboardInterrupt: 